In [8]:
from keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import sys

def preprocessDataset(train_text):
  
       
        #word tokenization using text-to-word-sequence
        train_text= str(train_text)
        tokenized_train_set = text_to_word_sequence(train_text,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
            
        #stop word removal
        stop_words = set(stopwords.words('english'))
        stopwordremove = [i for i in tokenized_train_set if not i in stop_words]
            
        
        #join words into sentence
        stopwordremove_text = ' '.join(stopwordremove)
            
            
        #remove numbers
        numberremove_text = ''.join(c for c in stopwordremove_text if not c.isdigit())
        
            
        #--Stemming--
        stemmer= PorterStemmer()

        stem_input=nltk.word_tokenize(numberremove_text)
        stem_text=' '.join([stemmer.stem(word) for word in stem_input])
            
            
        lemmatizer = WordNetLemmatizer()

        def get_wordnet_pos(word):
            """Map POS tag to first character lemmatize() accepts"""
            tag = nltk.pos_tag([word])[0][1][0].upper()
            tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}

            return tag_dict.get(tag, wordnet.NOUN)

        lem_input = nltk.word_tokenize(stem_text)
        lem_text= ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in lem_input])
            
        return lem_text


In [1]:
from sklearn.preprocessing import LabelEncoder

def encode_categorical(column):
    le = LabelEncoder()
    le.fit(column)
    encoded = le.transform(column)
    return encoded

In [2]:
import pandas  as pd
df=pd.read_csv("data\BBC News Train.csv")

In [3]:
df.columns

Index(['ArticleId', 'Text', 'Category'], dtype='object')

In [5]:
category=encode_categorical(df["Category"])

In [6]:
category

array([0, 0, 0, ..., 0, 4, 4])

In [12]:
df['Clean_Text'] = df['Text'].apply(preprocessDataset)

In [38]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf_vect = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english') #tfidfVectorizer
X = tfidf_vect.fit_transform(df["Clean_Text"])


In [33]:
category.shape

(1490,)

In [32]:
X.shape

(1490, 8544)

In [43]:
import numpy as np
np.c_[X.toarray(), category]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 4.],
       [0., 0., 0., ..., 0., 0., 4.]])